<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Sushant/DLS_Assignment_3_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 3 Part 3
### This assignment was discussed with Tejasram Ramesh and Sylvia B
### References Used:
  #### 1) https://towardsdatascience.com/audio-deep-learning-made-simple-part-1-state-of-the-art-techniques-da1d3dff2504
  #### 2) https://towardsdatascience.com/audio-deep-learning-made-simple-part-2-why-mel-spectrograms-perform-better-aad889a93505
  #### 3) https://www.youtube.com/watch?v=3gzI4Z2OFgY&t=1172s
  #### 4) https://www.youtube.com/watch?v=r6Ijqo5E3I4
  #### 5) Deep Learning with Python - by Francois Cholet
  #### 6) https://github.com/ShinoharaYuuyoru/NoiseReductionUsingGRU


In [1]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.1 MB/s 


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
import librosa
import librosa.display
import timeit
import os
from IPython.display import Audio
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import soundfile as sf
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling1D,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten
from keras.models import Model
import pickle
import tarfile

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.7454963840000346
GPU (s):
0.039328717999978835
GPU speedup over CPU: 44x


#### 1) 1200 noisy speech signals (from trx0000.wav to trx1199.wav)
#### 2) 120 clean speech signals (10sentences per speaker for 12 speakers)
#### 3) Original Clean Speech - (from trs0000.wav to trs1199.wav)
#### 4) Noise Sources - (from trn0000.wav to trn1199.wav)
#### 5) Adding these clean and noise speeches we get trx speeches but since it is given in the folder, there is no need to do it

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path_train = '/content/drive/MyDrive/DLS_Assignments/Assignment3_Data/timit-homework/tr/'
path_validation = '/content/drive/MyDrive/DLS_Assignments/Assignment3_Data/timit-homework/v/'
path_test = '/content/drive/MyDrive/DLS_Assignments/Assignment3_Data/timit-homework/te/'

In [8]:
!ls /content/drive/MyDrive/DLS_Assignments/Assignment3_Data/

timit-homework


In [9]:
def load_data(path):
  x=[]
  clean=[]
  noise=[]
  for file in os.listdir(path):
    if file.startswith('trx') or file.startswith('tex') or file.startswith('vx'):
      #if file not in x:
        x.append(file)

    elif file.startswith('trs') or file.startswith('tes') or file.startswith('vs'):
      #if file not in clean:
        clean.append(file)

    elif file.startswith('trn') or file.startswith('ten') or file.startswith('vn'):
      #if file not in noise:
        noise.append(file)
  
  x.sort()
  clean.sort()
  noise.sort()

  return x,clean,noise

### Loading the Train Data:

In [10]:
x_train, clean_train, noise_train = load_data(path_train)

In [11]:
print("The size of x_train is : ",len(x_train))
print("The size of clean_train is : ",len(clean_train))
print("The size of noise_train is : ",len(noise_train))

The size of x_train is :  1200
The size of clean_train is :  1200
The size of noise_train is :  1200


### Loading the Validation Data:

In [12]:
x_val, clean_val, noise_val = load_data(path_validation)

In [13]:
print("The size of x_val is : ",len(x_val))
print("The size of clean_val is : ",len(clean_val))
print("The size of noise_val is : ",len(noise_val))

The size of x_val is :  1200
The size of clean_val is :  1200
The size of noise_val is :  1200


### Loading the Test Data:

In [14]:
x_test, clean_test, noise_test = load_data(path_test)

In [15]:
print("The size of x_test is : ",len(x_test))
print("The size of clean_test is : ",len(clean_test))
print("The size of noise_test is : ",len(noise_test))

The size of x_test is :  400
The size of clean_test is :  0
The size of noise_test is :  0


In [16]:
def compute_stft(path,input):
  data_abs=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    pad = np.zeros((513, 178))
    pad[:, :S.shape[1]] = S
    pad=pad.T
    pad_abs = np.abs(pad)
    data_abs.append(pad_abs)
  return data_abs

### Computing STFT for Training Data

In [ ]:
x_train_stft = compute_stft(path_train,x_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [ ]:
print("Shape of x_train_stft is : ",x_train_stft[0].shape)
print(len(x_train_stft))

In [25]:
clean_train_stft = compute_stft(path_train,clean_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [26]:
print(len(clean_train_stft))

1200


In [27]:
noise_train_stft = compute_stft(path_train,noise_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


### Computing STFT for Validation Data

In [30]:
x_val_stft = compute_stft(path_validation,x_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [31]:
clean_val_stft = compute_stft(path_validation,clean_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [32]:
noise_val_stft = compute_stft(path_validation,noise_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


### Creating IBM Matrix for Training Set

In [28]:
ibm_train = [(clean_train_stft[i] > noise_train_stft[i]).astype(int) for i in range(len(clean_train_stft))]

In [29]:
ibm_train = np.stack(ibm_train)
noise_train_new = np.stack(noise_train_stft)

### Creating IBM Matrix for Validation Set

In [33]:
ibm_val = [(clean_val_stft[i] > noise_val_stft[i]).astype(int) for i in range(len(clean_val_stft))]

In [34]:
ibm_val = np.stack(ibm_val)
noise_val_new = np.stack(noise_val_stft)

In [35]:
ibm_val.shape

(1200, 178, 513)

In [36]:
model = tf.keras.Sequential();
model.add(layers.GRU(513, return_sequences=True))
model.add(layers.Dropout(rate = 0.2))
model.add(layers.GRU(513, return_sequences=True))
model.add(layers.Dropout(rate = 0.2))
model.add(layers.Dense(513, activation='sigmoid'))

In [37]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.BinaryCrossentropy())

In [38]:
history_1 = model.fit(noise_train_new, ibm_train, epochs=20, batch_size=10)

Epoch 1/20
120/120 [==============================] - 9s 38ms/step - loss: 0.4752
Epoch 2/20
120/120 [==============================] - 4s 37ms/step - loss: 0.3827
Epoch 3/20
120/120 [==============================] - 4s 37ms/step - loss: 0.3510
Epoch 4/20
120/120 [==============================] - 5s 38ms/step - loss: 0.3250
Epoch 5/20
120/120 [==============================] - 5s 38ms/step - loss: 0.3161
Epoch 6/20
120/120 [==============================] - 5s 38ms/step - loss: 0.3091
Epoch 7/20
120/120 [==============================] - 5s 38ms/step - loss: 0.3078
Epoch 8/20
120/120 [==============================] - 5s 38ms/step - loss: 0.3037
Epoch 9/20
120/120 [==============================] - 5s 38ms/step - loss: 0.3006
Epoch 10/20
120/120 [==============================] - 5s 39ms/step - loss: 0.2951
Epoch 11/20
120/120 [==============================] - 5s 39ms/step - loss: 0.2995
Epoch 12/20
120/120 [==============================] - 5s 39ms/step - loss: 0.2978
Epoch 13/20
1

In [39]:
model_path = '/content/drive/MyDrive/DLS_Assignments/Models/assign3_part3_model.h5'
model.save(model_path)

### Checking the model on Validation Data

In [40]:
model.evaluate(noise_val_new, ibm_val, batch_size=10)

120/120 [==============================] - 3s 18ms/step - loss: 0.3042


0.3042108416557312

In [41]:
def snr(clean, recon):
    print(clean.shape)
    recon = recon[:clean.shape[0]]
    return 10*np.log10(np.sum(clean**2)/np.sum(clean - recon)**2)

### Predicted IBM Mask

In [42]:
model_pred = model.predict(noise_val_new)

38/38 [==============================] - 2s 27ms/step


In [43]:
def compute_stft_2(path,input):
  data_abs=[]
  data=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    pad = np.zeros((513, 178))
    pad[:, :S.shape[1]] = S
    pad=pad.T
    pad_abs = np.abs(pad)
    data.append(pad)
    data_abs.append(pad_abs)
  return data,data_abs

In [44]:
def compute_stft_3(path,input):
  data_abs=[]
  data=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    pad = np.zeros((513, 178))
    pad[:, :S.shape[1]] = S
    pad=pad.T
    pad_abs = np.abs(pad)
    data.append(S)
    data_abs.append(pad_abs)
  return data,data_abs

In [45]:
noise_val_data,noise_val_new = compute_stft_2(path_validation,noise_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [46]:
clean_val_data,clean_val_new = compute_stft_3(path_validation,clean_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [47]:
noise_val_data = np.stack(noise_val_data)

### Generating Recovered Validation Sequence from Predicted IBM Mask

In [48]:
clean_pred = np.multiply(model_pred, noise_val_data)

In [49]:
noise_val_data.shape

(1200, 178, 513)

In [50]:
clean_val_stft[0]

array([[0.00812184, 0.00592763, 0.00213551, ..., 0.00017014, 0.00010148,
        0.00033313],
       [0.00248497, 0.00062856, 0.00258446, ..., 0.00020294, 0.00033425,
        0.0003165 ],
       [0.00805179, 0.00287515, 0.00200844, ..., 0.00023642, 0.00038201,
        0.00047942],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Calculate SNR

In [51]:
snr = []
for i in range(len(clean_val_data)):
    S_test = librosa.istft(clean_val_data[i], hop_length=512)
    S_test_2 = librosa.istft(clean_pred[i].T, hop_length=512)
    s = sum((S_test)**2)
    S_test_2 = S_test_2[:S_test.shape[0]]
    #diff_st = sum(np.subtract(S_test,S_test_2)**2)+1e-20
    SNR = 10*np.log10(np.sum(S_test**2)/np.sum(S_test - S_test_2)**2)
    snr.append(SNR)

In [52]:
snr

[16.946794189640116,
 16.05458925064433,
 14.310728844947544,
 24.729587103531113,
 13.535351963547948,
 17.4992976388433,
 15.426731873722687,
 19.64998485811929,
 17.84081608685343,
 15.122927197673663,
 16.127348625874426,
 19.334466953271228,
 31.274199752945933,
 18.87814313386005,
 16.346656931042503,
 16.372710966740552,
 13.490830562041953,
 18.942830730663665,
 14.81867796667821,
 16.336498548198875,
 24.523052110078435,
 21.763299172528416,
 32.48373410069276,
 19.348330055150274,
 28.118276467101396,
 19.238890809325177,
 17.287838114810697,
 18.987797419008565,
 27.202261203975194,
 26.579616718692918,
 23.375966317901824,
 24.51305975345907,
 35.18419839228092,
 23.90834519147154,
 31.71090485748474,
 26.630658401102473,
 18.377802095106887,
 23.88285598669942,
 30.825630520595062,
 21.106444613954253,
 23.997006977651303,
 21.633699336737422,
 18.10950510618637,
 21.220750379679703,
 17.672969969586525,
 20.959713423243578,
 26.12175710437366,
 20.687582586181847,
 23.173

In [53]:
sum(snr)/1200

20.511831676371767

In [54]:
print("Average SNR is : " ,sum(snr)/1200)

Average SNR is :  20.511831676371767


### Checking for the Test data set

In [55]:
def compute_stft_4(path,input):
  data_abs=[]
  data=[]
  for i in input:
    s, sr = librosa.load(path+i, sr=None)
    S = librosa.stft(s, n_fft=1024, hop_length=512)
    pad = np.zeros((513, 195))
    pad[:, :S.shape[1]] = S
    pad=pad.T
    pad_abs = np.abs(pad)
    data.append(pad)
    data_abs.append(pad_abs)
  return data,data_abs

In [56]:
noise_test_data,noise_test_new = compute_stft_4(path_test,x_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [57]:
noise_test_new = np.stack(noise_test_new)

In [60]:
model_2 = keras.models.load_model('/content/drive/MyDrive/DLS_Assignments/Models/assign3_part3_model.h5')

### Predicted IBM Mask for the Test Set

In [61]:
model_pred_test = model.predict(noise_test_new, batch_size=10)

40/40 [==============================] - 2s 21ms/step


In [62]:
noise_test_data = np.stack(noise_test_data)

### Generating Clean Test Sequence from Predicted IBM Mask

In [63]:
clean_pred_test = np.multiply(model_pred_test, noise_test_data)

### Loading the Reconstructed Signals on the given path

In [65]:
# snr = []
# for i in range(len(clean_val_data)):
#     S_test = librosa.istft(clean_val_data[i], hop_length=512)
#     S_test_2 = librosa.istft(clean_pred[i].T, hop_length=512)
#     s = sum((S_test)**2)
#     S_test_2 = S_test_2[:S_test.shape[0]]
#     #diff_st = sum(np.subtract(S_test,S_test_2)**2)+1e-20
#     SNR = 10*np.log10(np.sum(S_test**2)/np.sum(S_test - S_test_2)**2)
#     snr.append(SNR)

out_path = '/content/drive/MyDrive/DLS_Assignments/Assignment3_Data/timit-homework/output/'

for i in range(len(x_test)):
  test_s , test_sr = librosa.load(path_test+x_test[i], sr=None)
  test_out = librosa.istft(clean_pred_test[i].T, win_length=1024, hop_length=512)
  test_out = test_out[:test_s.shape[0]]
  sf.write(out_path+x_test[i]+"_cleaned.wav",test_out,test_sr)